In [4]:
# First initialize datasets. These are used for training and evaluating the model.

from pathlib import Path
from random import sample

from fortuna import ObjectDetectionDataset
from fortuna.data import pedestrians_dirpath
import fortuna.torchutils.transforms as T

# Specify paths to images and masks.
image_paths = sorted((pedestrians_dirpath / "Images").rglob("*"))
mask_paths = sorted((pedestrians_dirpath / "Masks").rglob("*"))

# We're going to use a random subsample of the dataset to validate the model.
n = 10 # len(image_paths)
training_fraction = 0.9
training_ix = sample(range(n), int(training_fraction*n))
validation_ix = list(set(range(n)) - set(training_ix))

# Create ObjectDetectionDatasets for training and validation.
training_dataset = ObjectDetectionDataset(
    image_paths=[image_paths[j] for j in training_ix],
    mask_paths=[mask_paths[j] for j in training_ix],
    transforms=[T.RandomHorizontalFlip(0.5)]
)
validation_dataset = ObjectDetectionDataset(
    image_paths=[image_paths[j] for j in validation_ix],
    mask_paths=[mask_paths[j] for j in validation_ix]
)

In [5]:
# Initialize and fit the object detector.

from fortuna import ObjectDetectionModel
from fortuna.data import pedestrians_dirpath

model = ObjectDetectionModel(n_classes=2)

model.fit(
    training_dataset,
    validation_dataset
)

Epoch: [0]  [0/5]  eta: 0:00:42  lr: 0.000251  loss: 3.3684 (3.3684)  loss_classifier: 0.6806 (0.6806)  loss_box_reg: 0.2409 (0.2409)  loss_mask: 1.8763 (1.8763)  loss_objectness: 0.5563 (0.5563)  loss_rpn_box_reg: 0.0143 (0.0143)  time: 8.5074  data: 0.0236
Epoch: [0]  [4/5]  eta: 0:00:05  lr: 0.001000  loss: 1.3856 (1.9998)  loss_classifier: 0.3450 (0.3878)  loss_box_reg: 0.1434 (0.1452)  loss_mask: 1.0227 (1.2392)  loss_objectness: 0.0379 (0.2182)  loss_rpn_box_reg: 0.0089 (0.0094)  time: 5.7571  data: 0.0215
Epoch: [0] Total time: 0:00:28 (5.7576 s / it)


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


TypeError: 'NoneType' object is not iterable

In [ ]:
# Evaluate the detector.

model.evaluate(
    validation_dataset
)

In [ ]:
# Export model.

model.write("path_to_model_file.onnx") # Export as e.g. ONNX model.